# USING OPENAI API KEYS

LOADING THE PACKAGES

In [ ]:
#%pip install langchain
#%pip install faiss-cpu
#%pip install openai

In [ ]:
%pip install langchain unstructured
%pip install tiktoken


DATASET PREPARATION

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "#####"

In [ ]:
urls = [
    'https://www.medicalnewstoday.com/articles/73936'
    'https://www.totalhealthandfitness.com/what-is-mindful-eating-your-complete-guide-to-enjoying-your-food-more/'
    'https://www.totalhealthandfitness.com/nutrition-and-chronic-disease/'
    'https://www.glofox.com/blog/fitness-blogs/'
]

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
loaders = UnstructuredURLLoader(urls=urls)
data = loaders.load()

In [ ]:
data

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n', 
                                      chunk_size=1000, 
                                      chunk_overlap=200)


docs = text_splitter.split_documents(data)

In [ ]:
docs

In [ ]:
len(docs)

In [ ]:
import pickle
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
# RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

'''vectorStore_openAI = FAISS.from_documents(docs, embeddings)

with open("faiss_store_openai.pkl", "wb") as f:
     pickle.dump(vectorStore_openAI, f)

with open("faiss_store_openai.pkl", "rb") as f:
    VectorStore = pickle.load(f)'''



In [ ]:
# VectorStore

MODEL FINE-TUNING
(GPT-3)

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
from langchain_community.chat_models import ChatOpenAI
llm=OpenAI(temperature=0, model_name='gpt-3.5-turbo-0125')

In [ ]:
llm

CHATBOT INTERFACE & TESTING

In [ ]:
# chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=VectorStore.as_retriever())

In [ ]:
# chain({"question": "how to maintain a healthy diet?"}, return_only_outputs=True)

API ENDPOINTS

In [ ]:
#%pip install requests

In [ ]:
import requests

In [ ]:
def Questions(query):
    url = "http://localhost:11434/api/generate"
    params = {"query":query}
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    
    else:
        print("Ollama API request failed.")
    

In [ ]:
def Answer(question):
    answer = chain({"questions":question}, return_only_outputs=True)
    
    return answer

In [ ]:
# Not able to load 'chain' because of the RateLimitError

'''question = "how to maintain healthy diet?"
answer = Answer(question)
print(answer)'''